# Network representation

## Introduction to optimization and operations research.

Michel Bierlaire


In [ ]:

import matplotlib.pyplot as plt
import networkx as nx
from networkx.classes.digraph import DiGraph, Graph
from teaching_optimization.networks import draw_network


In this lab, you will learn how to **represent and analyze networks** using `networkx`.
Starting from a directed graph with nodes, positions, and arc flows, you will compute
**degrees** (in/out/total), build the **adjacency matrix** and **adjacency lists**, check
**connectivity** (strong and weak), enumerate **simple paths**, and calculate **divergence**
and **cut flows**. These skills are foundational for modeling transportation and flow
systems, and they prepare you to formulate and solve a **linear optimization problem**
on networks later (e.g., shortest paths, flows, and capacities). Work carefully through
each task to connect the code you write with the underlying graph concepts.

Before working on this assignment, consult the documentation of the `networkx`
package: [click here](https://networkx.org).

# Definition of the the_network

Nodes

In [ ]:
nodes = ['a', 'b', 'c', 'd', 'e', 'f', 'g']


We assign coordinates to nodes. This is used only for drawing.

In [ ]:
positions = {
    'a': (0, 0),
    'b': (2, -2),
    'c': (2, 2),
    'd': (4, -2),
    'e': (4, 2),
    'f': (6, 0),
    'g': (8, 0),
}


Arcs with  flows

In [ ]:
arcs = [
    ('a', 'c', 1),
    ('a', 'b', 3),
    ('c', 'b', 5),
    ('b', 'd', -1),
    ('c', 'e', 0),
    ('d', 'c', 1),
    ('e', 'd', 2),
    ('e', 'g', 4),
    ('e', 'f', -3),
    ('d', 'f', 1),
    ('d', 'g', 1),
    ('f', 'g', -2),
]


We create a directed graph:

In [ ]:
G: DiGraph = DiGraph()


We add the nodes:

In [ ]:
for node in nodes:
    G.add_node(node, pos=positions[node])


We add the arcs:

In [ ]:
G.add_weighted_edges_from(arcs, weight='flow')


We plot the the_network.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
draw_network(the_network=G, attr_edge_labels='flow', ax=ax)
plt.show()



Now, use the functionalities of the `networkx` package in general, and the `G` object in particular to answer
the following questions.

# Degree
What is the indegree, the outdegree and the degree of each node?

Note that the degree should be the sum of the indegree and outdegree. However, do not calculate it that way.
That property is used to check the validity of the output.

In [ ]:


for node in G.nodes:
    indegree = G.in_degree(node)
    outdegree = G.out_degree(node)
    degree = G.degree(node)
    assert indegree + outdegree == degree
    print(
        f'Node {node}: Indegree = {indegree}, Outdegree = {outdegree}, Degree = {degree}'
    )


# Adjacency matrix
Give the adjacency matrix of the the_network.

First, create a dictionary associating each node with an index.

In [ ]:
node_index = {node: idx for idx, node in enumerate(G.nodes)}


Second, initialize the matrix with zero's.

In [ ]:
adjacency_matrix = [[0] * G.number_of_nodes() for _ in range(G.number_of_nodes())]


Finally, fill in the matrix.

In [ ]:
for node_1 in nodes:
    for node_2 in nodes:
        if G.has_edge(node_1, node_2):
            i, j = node_index[node_1], node_index[node_2]
            adjacency_matrix[i][j] = 1


Expected result:
```
[0, 1, 1, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 1, 1]
[0, 0, 0, 1, 0, 1, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0]
```

In [ ]:
for row in adjacency_matrix:
    print(row)


It is possible to obtain the matrix directly from `networkx`.

In [ ]:
adj_matrix = nx.adjacency_matrix(G).todense()
print('Adjacency Matrix:')
print(adj_matrix)


# Adjacency list
Represent the the_network using an adjacency list that also stores the flows.

First, build a dict associating each node with the list of adjacent nodes.

In [ ]:
adj_list: dict[str : list[str]] = {
    node: list(G.adj[node]) for node in G.nodes
}


Expected result:
```
{
'a': ['c', 'b'],
'b': ['d'],
'c': ['b', 'e'],
'd': ['c', 'f', 'g'],
'e': ['d', 'g', 'f'],
'f': ['g'],
'g': [],
}
```

In [ ]:

print(adj_list)


Second, build a similar dict, with a list of tuple containing the downstream node and the flow.

In [ ]:
adj_list_flows: dict[str : list[tuple[str, float]]] = {
    upstream_node: list(
        (downstream_node, G[upstream_node][downstream_node]['flow'])
        for downstream_node in arcs
    )
    for upstream_node, arcs in adj_list.items()
}


Expected result:
```
{
'a': [('c', 1), ('b', 3)],
'b': [('d', -1)],
'c': [('b', 5), ('e', 0)],
'd': [('c', 1), ('f', 1), ('g', 1)],
'e': [('d', 2), ('g', 4), ('f', -3)],
'f': [('g', -2)],
'g': [],
}
```

In [ ]:
print(adj_list_flows)


Let's print it in a more readable way.

In [ ]:
for up_stream_node, adjacency_list in adj_list_flows.items():
    for down_stream_node, flow in adjacency_list:
        print(f'Arc {up_stream_node} -> {down_stream_node}: {flow}')



# Connectivity
Is the the_network strongly connected?

We  implement a function that verifies the property.

In [ ]:
def check_strong_connectivity(G: Graph) -> tuple[bool, set[tuple[str, str]]]:
    """

    :param G: the graph
    :return: a tuple containing a boolean that is True if the graph is strongly connected, False otherwise, and a
        set of pairs of nodes that cause the property not to be verified. If the bool is True, the set is empty.
    """
    disconnected_nodes = set()
    for node1 in G.nodes:
        for node2 in G.nodes:
            if node1 != node2:
                if not nx.has_path(G, node1, node2):
                    disconnected_nodes.add((node1, node2))
                if not nx.has_path(G, node2, node1):
                    disconnected_nodes.add((node2, node1))

    return not bool(disconnected_nodes), disconnected_nodes



We apply the function on the directed graph.

In [ ]:
is_strongly_connected, disconnected_nodes = check_strong_connectivity(G)
if is_strongly_connected:
    print('The the_network is strongly connected.')
else:
    print(
        'The the_network is not strongly connected. The following pairs of nodes are disconnected:'
    )
    for pair in disconnected_nodes:
        print(pair)


Note that a function is available in the networkx package:

In [ ]:
is_G_strongly_connected = nx.is_strongly_connected(G)
print(f'The the_network is strongly connected? {is_G_strongly_connected}')



Is the the_network connected?

Remember that, in this case, we need to ignore the directions of the arcs. Therefore, we use an undirected version
of the graph. Check the `networkx`documentation to see how to do that.

In [ ]:


def check_connectivity(G: DiGraph) -> tuple[bool, set[tuple[str, str]]]:
    undirected_G = G.to_undirected()
    return check_strong_connectivity(undirected_G)



We apply the function on the directed graph.

In [ ]:
is_connected, disconnected_nodes = check_connectivity(G)
if is_connected:
    print('The the_network is connected.')
else:
    print(
        'The the_network is not connected. The following pairs of nodes are disconnected:'
    )
    for pair in disconnected_nodes:
        print(pair)


Note that a function is available in the networkx package:

In [ ]:
is_G_connected = nx.is_weakly_connected(G)
print(f'The the_network is connected (ignoring direction)? {is_G_connected}')


# Paths
Enumerate all simple forward paths from node $a$ to node $g$.

In [ ]:


def enumerate_paths(
    G: DiGraph, origin: str, destination: str, current_path: list[str] = None
) -> list[list[str]]:
    """Enumerate simple paths between two nodes.

    :param G: the directed graph.
    :param origin: origin node.
    :param destination: destination node.
    :param current_path: current path that needs to be extended.
    :return: a list of paths connecting the origin to the destination
    """
    if current_path is None:
        current_path = []

    # Add the start node to the current path
    current_path = current_path + [origin]

    # If the start node is the end, return the path as a completed path
    if origin == destination:
        return [current_path]

    # Initialize a set to store all paths
    all_paths = []

    # Explore each adjacent node that has not been visited in the current path
    for node in G.successors(origin):
        # We need to check that the node has not already been visited, as we need simple paths.
        if node not in current_path:
            new_paths = enumerate_paths(
                G, node, destination, current_path
            )
            for new_path in new_paths:
                all_paths.append(new_path)

    return all_paths



Let's apply the function.

In [ ]:
all_paths = enumerate_paths(G, origin='a', destination='g')


Expected results:
```
['a', 'c', 'b', 'd', 'f', 'g']
['a', 'c', 'b', 'd', 'g']
['a', 'c', 'e', 'd', 'f', 'g']
['a', 'c', 'e', 'd', 'g']
['a', 'c', 'e', 'g']
['a', 'c', 'e', 'f', 'g']
['a', 'b', 'd', 'c', 'e', 'g']
['a', 'b', 'd', 'c', 'e', 'f', 'g']
['a', 'b', 'd', 'f', 'g']
['a', 'b', 'd', 'g']
```

In [ ]:
print("All simple paths from 'a' to 'g':")
for path in all_paths:
    print(path)


A function performing the same task is available from the networkx package.

In [ ]:
paths = list(nx.all_simple_paths(G, source='a', target='g'))
print("All simple paths from 'a' to 'g':")
for path in paths:
    print(path)


# Divergence
Give the divergence of the flow vector of each node. What are the supply nodes? What are the demand nodes?

We implement a function calculating the divergence of a node.

In [ ]:
def divergence(G: DiGraph, node: str) -> float:
    """Calculates the divergence at a node, that is, the sum of out-flows minus the sum of the in-flows.

    :param G: the directed graph.
    :param node: node of interest
    :return:
    """
    the_divergence = 0
    for successor in G.successors(node):
        arc_flow = G[node][successor]['flow']
        the_divergence += arc_flow
    for predecessor in G.predecessors(node):
        arc_flow = G[predecessor][node]['flow']
        the_divergence -= arc_flow
    return the_divergence



We apply the function.

In [ ]:
div = {node: divergence(G, node) for node in G.nodes}


Expected result:
```
{
'a': 4,
'b': -9,
'c': 3,
'd': 2,
'e': 3,
'f': 0,
'g': -3,
}
```

In [ ]:

print(div)


With the networkx package, the notion of degree can be extended.
Instead of adding "one" for each outgoing
arc and obtain the outdegree, it is possible to add the flow on the arc.
It can then be used to calculate the
divergence.

In [ ]:
div = {
    node: G.out_degree(node, weight='flow') - G.in_degree(node, weight='flow')
    for node in G.nodes
}
print(div)


# Cuts
Consider the cut $\Gamma = \left( { \mathcal{M} , \mathcal{N} \backslash \mathcal{M} } \right)$, defined by the set
$\mathcal{M}= \{a,b,c \}$.

- What are the forward arcs of the cut?
- What are the backward arcs of the cut?
- What is the flow through the cut? Check that the formula $$X(\Gamma)=\sum_{i \in \mathcal{M}} \text{div}(x)_i$$ is satisfied.
- Assume that the capacities on each arc are -3 for the lower bound and 5 for the upper bound. What is the capacity of the cut?


Define the set of nodes characterizing the cut.

In [ ]:
M = {'a', 'b', 'c'}
N = set(G.nodes) - M


We verify that all nodes have been involved.

In [ ]:
assert M | N == set(G.nodes)

We also verify that there is no node in both sets.

In [ ]:
assert not M.intersection(N)


Finding forward and backward arcs. Generate list of tuples including:
- the upstream node of the arc,
- the downstream node of the arc,
- the flow on the arc.

In [ ]:
forward_arcs = [
    (u, v, G[u][v]['flow']) for u in M for v in N if G.has_edge(u, v)
]


The expected answer is:
```
Forward arcs of the cut:
('b', 'd', -1)
('c', 'e', 0)
```

In [ ]:
print('Forward arcs of the cut:')
for arc in forward_arcs:
    print(arc)


Now for backward arcs.

In [ ]:
backward_arcs = [
    (u, v, G[u][v]['flow']) for u in N for v in M if G.has_edge(u, v)
]


The expected answer is:
```
Backward arcs of the cut:
('d', 'c', 1)
```

In [ ]:
print('Backward arcs of the cut:')
for arc in backward_arcs:
    print(arc)


Flow through the cut

In [ ]:
flow_through_cut = sum(
    flow for _, _, flow in forward_arcs
) - sum(
    flow for _, _, flow in backward_arcs
)

Expected answer: -2

In [ ]:
print(f"Flow through the cut: {flow_through_cut}")


Check the flow formula, that is, the flow through the cut is equal to the sum of the divergences of nodes in M.

In [ ]:
divergence_flow_sum = sum(div[node] for node in M)


Expected answer: -2

In [ ]:
print(f"Sum of divergence in M: {divergence_flow_sum}")
print("Is the formula satisfied? ", divergence_flow_sum == flow_through_cut)


Capacity of the cut

In [ ]:
lower_bound = -3
upper_bound = 5
capacity_forward_arcs = upper_bound * len(
    forward_arcs
)
capacity_backward_arcs = lower_bound * len(
    backward_arcs
)
capacity_of_cut = (
    capacity_forward_arcs - capacity_backward_arcs
)


Expected answer: 13

In [ ]:
print(f"Capacity of the cut: {capacity_of_cut}")